In [51]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [4]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
train_df.set_index('id', inplace=True)
train_df.head()

,Unnamed: 0,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,monthly_revenue
id,,,,,,,,,,,,,,,,,,,,,
19792418,879,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,f,3,...,4.80,4.82,4.90,4.87,4.69,4.81,f,3,0.77,2108
1015650685503221866,6416,Guest suite in Vancouver · ★New · 2 bedrooms ·...,NaN,139792573,Daniel,within a few hours,100%,100%,f,1,...,NaN,NaN,NaN,NaN,NaN,NaN,f,1,NaN,2730
35265562,1820,Guest suite in Vancouver · ★4.85 · 2 bedrooms ...,Beautiful neighbourhood close to prosperous Ma...,265504225,Alex,within an hour,100%,98%,t,1,...,4.90,4.78,4.97,4.94,4.90,4.75,f,1,3.22,2254
911948980885194155,5346,Home in Vancouver · ★5.0 · 1 bedroom · 1 bed ·...,We are located in a quiet residential neighbor...,22595056,Raymond,NaN,NaN,92%,t,1,...,5.00,5.00,5.00,5.00,4.86,5.00,f,1,1.28,3187
46069251,2484,Guest suite in Vancouver · ★4.93 · 1 bedroom ·...,Kitsilano at it's best! Short walk to all the ...,65683877,Yendi,within an hour,100%,95%,t,2,...,4.93,4.89,4.97,4.97,4.96,4.85,f,1,2.01,3479


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5352 entries, 19792418 to 19298482
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      5352 non-null   int64  
 1   name                            5352 non-null   object 
 2   neighborhood_overview           3476 non-null   object 
 3   host_id                         5352 non-null   int64  
 4   host_name                       5352 non-null   object 
 5   host_response_time              4263 non-null   object 
 6   host_response_rate              4263 non-null   object 
 7   host_acceptance_rate            4620 non-null   object 
 8   host_is_superhost               5315 non-null   object 
 9   host_listings_count             5352 non-null   int64  
 10  host_total_listings_count       5352 non-null   int64  
 11  neighbourhood                   3476 non-null   object 
 12  neighbourhood_cleansed      

In [6]:
train_df.drop(train_df.columns[0], axis=1, inplace=True)
train_df.drop('bathrooms', axis=1, inplace=True)
train_df.drop('bedrooms', axis=1, inplace=True)

In [7]:
missing_values = train_df.isnull().sum()
print(missing_values[missing_values > 0].sort_values(ascending=False))  # Columns with missing data

neighborhood_overview          1876
neighbourhood                  1876
host_response_time             1089
host_response_rate             1089
reviews_per_month               906
review_scores_checkin           901
review_scores_location          901
review_scores_value             901
review_scores_rating            900
review_scores_accuracy          900
review_scores_cleanliness       900
review_scores_communication     900
host_acceptance_rate            732
price                           665
host_is_superhost                37
beds                             36
dtype: int64


In [8]:
train_df.columns

Index(['name', 'neighborhood_overview', 'host_id', 'host_name',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'neighbourhood', 'neighbourhood_cleansed', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates', 'beds', 'amenities',
       'price', 'minimum_nights', 'maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count', 'reviews_per_month',
       'monthly_revenue'],
      dtype='object')

In [9]:
train_df['neighbourhood_cleansed'].value_counts()

neighbourhood_cleansed
Downtown                    1389
West End                     411
Kitsilano                    381
Mount Pleasant               353
Kensington-Cedar Cottage     326
Downtown Eastside            299
Riley Park                   250
Grandview-Woodland           217
Hastings-Sunrise             216
Renfrew-Collingwood          190
Dunbar Southlands            171
Marpole                      158
Fairview                     144
Sunset                       143
Victoria-Fraserview          128
Oakridge                     107
West Point Grey              105
Arbutus Ridge                 87
Kerrisdale                    68
Killarney                     65
Shaughnessy                   63
South Cambie                  54
Strathcona                    27
Name: count, dtype: int64

In [10]:
for idx, i in enumerate(train_df['name']):
# for idx, i in enumerate(train_df['neighborhood_overview']):
    print(i)
    if idx == 5:
        break

Home in Vancouver · ★4.75 · 1 bedroom · 1 bed · 1 bath
Guest suite in Vancouver · ★New · 2 bedrooms · 2 beds · 1 bath
Guest suite in Vancouver · ★4.85 · 2 bedrooms · 3 beds · 1 bath
Home in Vancouver · ★5.0 · 1 bedroom · 1 bed · 1 bath
Guest suite in Vancouver · ★4.93 · 1 bedroom · 2 beds · 1 bath
Condo in Vancouver · ★4.88 · Studio · 1 bed · 1 bath


In [11]:
# Regex patterns for extraction
property_type_pattern = r"^(.*?) in Vancouver"
rating_pattern = r"★([0-9.]+|New)"
bedrooms_pattern = r"(\d+) bedroom"
beds_pattern = r"(\d+) bed"
baths_pattern = r"(\d+) bath"

# Functions to extract each feature
def extract_property_type(name):
    match = re.search(property_type_pattern, name)
    return match.group(1) if match else None

def extract_rating(name):
    match = re.search(rating_pattern, name)
    return float(match.group(1)) if match and match.group(1).replace('.', '', 1).isdigit() else None

def extract_bedrooms(name):
    match = re.search(bedrooms_pattern, name)
    return int(match.group(1)) if match else (0 if 'Studio' in name else None)

def extract_beds(name):
    match = re.search(beds_pattern, name)
    return int(match.group(1)) if match else None

def extract_baths(name):
    match = re.search(baths_pattern, name)
    return int(match.group(1)) if match else None

# Apply functions to extract features
train_df['property_type'] = train_df['name'].apply(extract_property_type)
train_df['rating'] = train_df['name'].apply(extract_rating)
train_df['bedrooms'] = train_df['name'].apply(extract_bedrooms)
train_df['beds'] = train_df['name'].apply(extract_beds)
train_df['baths'] = train_df['name'].apply(extract_baths)

# Fill missing ratings for "New" as a separate indicator
train_df['is_new_listing'] = train_df['rating'].isnull().astype(int)
train_df['rating'] = train_df['rating'].fillna(0)  # Optional: set "New" ratings to 0 or another placeholder

# Show resulting DataFrame
train_df.head()


,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,monthly_revenue,rating,bedrooms,baths,is_new_listing
id,,,,,,,,,,,,,,,,,,,,,
19792418,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. <br /><br />Hig...,57488206,Jessi,NaN,NaN,NaN,f,3,3,...,4.69,4.81,f,3,0.77,2108,4.75,1.0,1.0,0
1015650685503221866,Guest suite in Vancouver · ★New · 2 bedrooms ·...,NaN,139792573,Daniel,within a few hours,100%,100%,f,1,4,...,NaN,NaN,f,1,NaN,2730,0.00,2.0,1.0,1
35265562,Guest suite in Vancouver · ★4.85 · 2 bedrooms ...,Beautiful neighbourhood close to prosperous Ma...,265504225,Alex,within an hour,100%,98%,t,1,1,...,4.90,4.75,f,1,3.22,2254,4.85,2.0,1.0,0
911948980885194155,Home in Vancouver · ★5.0 · 1 bedroom · 1 bed ·...,We are located in a quiet residential neighbor...,22595056,Raymond,NaN,NaN,92%,t,1,1,...,4.86,5.00,f,1,1.28,3187,5.00,1.0,1.0,0
46069251,Guest suite in Vancouver · ★4.93 · 1 bedroom ·...,Kitsilano at it's best! Short walk to all the ...,65683877,Yendi,within an hour,100%,95%,t,2,3,...,4.96,4.85,f,1,2.01,3479,4.93,1.0,1.0,0


In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5352 entries, 19792418 to 19298482
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            5352 non-null   object 
 1   neighborhood_overview           3476 non-null   object 
 2   host_id                         5352 non-null   int64  
 3   host_name                       5352 non-null   object 
 4   host_response_time              4263 non-null   object 
 5   host_response_rate              4263 non-null   object 
 6   host_acceptance_rate            4620 non-null   object 
 7   host_is_superhost               5315 non-null   object 
 8   host_listings_count             5352 non-null   int64  
 9   host_total_listings_count       5352 non-null   int64  
 10  neighbourhood                   3476 non-null   object 
 11  neighbourhood_cleansed          5352 non-null   object 
 12  latitude                    

In [16]:
# Clean up HTML tags and fill NaN values
train_df['neighborhood_overview'] = train_df['neighborhood_overview'].fillna("No description provided")
train_df['neighborhood_overview'] = train_df['neighborhood_overview'].apply(lambda x: re.sub(r'<.*?>', '', str(x)))

# Keyword presence feature extraction
def has_keyword(text, keyword):
    return int(bool(re.search(r'\b' + keyword + r'\b', text.lower())))

# Apply keyword search for various categories
keywords = ['park', 'restaurant', 'shopping', 'beach', 'quiet', 'transit', 'lively', 'trendy']
for keyword in keywords:
    train_df[f'has_{keyword}'] = train_df['neighborhood_overview'].apply(lambda x: has_keyword(x, keyword))

# Sentiment Analysis
train_df['polarity'] = train_df['neighborhood_overview'].apply(lambda x: TextBlob(x).sentiment.polarity)
train_df['subjectivity'] = train_df['neighborhood_overview'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

# Text Length
train_df['review_length'] = train_df['neighborhood_overview'].apply(lambda x: len(x.split()))

# Show resulting DataFrame
train_df.head()


,name,neighborhood_overview,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,has_restaurant,has_shopping,has_beach,has_quiet,has_transit,has_lively,has_trendy,polarity,subjectivity,review_length
id,,,,,,,,,,,,,,,,,,,,,
19792418,Home in Vancouver · ★4.75 · 1 bedroom · 1 bed ...,Everything you need is nearby. Highlight: Walk...,57488206,Jessi,NaN,NaN,NaN,f,3,3,...,0,0,0,0,0,0,0,0.217857,0.621429,91
1015650685503221866,Guest suite in Vancouver · ★New · 2 bedrooms ·...,No description provided,139792573,Daniel,within a few hours,100%,100%,f,1,4,...,0,0,0,0,0,0,0,0.000000,0.000000,3
35265562,Guest suite in Vancouver · ★4.85 · 2 bedrooms ...,Beautiful neighbourhood close to prosperous Ma...,265504225,Alex,within an hour,100%,98%,t,1,1,...,0,1,0,0,0,0,0,0.108974,0.292308,124
911948980885194155,Home in Vancouver · ★5.0 · 1 bedroom · 1 bed ·...,We are located in a quiet residential neighbor...,22595056,Raymond,NaN,NaN,92%,t,1,1,...,0,0,0,1,0,0,0,0.258333,0.791667,52
46069251,Guest suite in Vancouver · ★4.93 · 1 bedroom ·...,Kitsilano at it's best! Short walk to all the ...,65683877,Yendi,within an hour,100%,95%,t,2,3,...,0,0,0,0,1,0,0,0.333333,0.200000,51


In [17]:
train_df['host_response_rate'] = train_df['host_response_rate'].str.strip('%').astype(float) / 100
train_df['host_response_rate'].fillna(0, inplace=True) 
train_df['host_acceptance_rate'] = train_df['host_acceptance_rate'].str.strip('%').astype(float) / 100
train_df['host_acceptance_rate'].fillna(0, inplace=True) 

/var/folders/mp/57zdplyn4kl5d5l8mxcwwknr0000gn/T/ipykernel_80906/1195840191.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['host_response_rate'].fillna(0, inplace=True)


## Train-test split

In [25]:
X = train_df.drop(columns = ['monthly_revenue', 'amenities','name', 'host_name'])
y = train_df['monthly_revenue']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [32]:
X_train.columns

Index(['neighborhood_overview', 'host_id', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'beds', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count', 'reviews_per_month', 'rating',
       'bedrooms', 'baths', 'is_new_listing', 'has_park', 'has_restaurant',
       'has_shopping', 'has_beach', 'has_quie

In [34]:
response_time_mapping = {
    "within an hour": 1,
    "within a few hours": 2,
    "within a day": 3,
    "within a week": 4
}
X_train['host_response_time'] = X_train['host_response_time'].map(response_time_mapping)

In [35]:
numeric_features = [
    'host_response_rate', 'host_acceptance_rate', 'host_listings_count', 'host_total_listings_count',
    'accommodates', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm', 'availability_30', 'availability_60', 'availability_90', 'availability_365',
    'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
    'review_scores_value', 'calculated_host_listings_count', 'reviews_per_month', 'rating', 'bedrooms', 'baths',
    'review_length', 'polarity', 'subjectivity', 'latitude', 'longitude'
]

categorical_features = ['property_type', 'room_type', 'neighbourhood', 'neighbourhood_cleansed']

ordinal_features = ['host_response_time'] 

binary_features = [
    'host_is_superhost', 'instant_bookable', 'is_new_listing', 'has_park', 'has_restaurant', 
    'has_shopping', 'has_beach', 'has_quiet', 'has_transit', 'has_lively', 'has_trendy'
]

## Column transform

In [43]:
numeric_transformer = StandardScaler()

In [47]:
ordinal_transformer = OrdinalEncoder(
    categories = [response_time_mapping], 
    dtype = int)

In [48]:
binary_transformer = OneHotEncoder(
    handle_unknown="ignore", 
    sparse_output=False, 
    drop='if_binary', 
    dtype = int
)

In [49]:
si = SimpleImputer(strategy="constant", fill_value="missing")

ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype = int)

categorical_transformer = make_pipeline(si, ohe)

In [52]:
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (ordinal_transformer, ordinal_features),
    (binary_transformer, binary_features),
    (categorical_transformer, categorical_features),
    #("drop", drop_features)
)

In [53]:
transformed_df = preprocessor.fit_transform(X_train)

ValueError: could not convert string to float: '$123.00'